In [ ]:
!pip install PyPaperBot

     |████████████████████████████████| 97 kB 3.1 MB/s 
     |████████████████████████████████| 220 kB 30.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 57 kB 5.1 MB/s 
     |████████████████████████████████| 829 kB 54.3 MB/s 
     |████████████████████████████████| 138 kB 48.5 MB/s 
     |████████████████████████████████| 103 kB 57.8 MB/s 
     |████████████████████████████████| 437 kB 42.3 MB/s 
     |████████████████████████████████| 503 kB 40.5 MB/s 
     |████████████████████████████████| 46 kB 4.2 MB/s 
     |████████████████████████████████| 743 kB 36.9 MB/s 
     |████████████████████████████████| 436 kB 19.8 MB/s 
     |████████████████████████████████| 436 kB 60.4 MB/s 
     |████████████████████████████████| 414 kB 49.4 MB/s 
     |████████████████████████████████| 413 kB 52.0 MB/s 
     |████████████████████████████████| 413 kB 45.5 MB/s 
     |████████████████████████████████| 392 kB 20.0 MB/s 
     |████████████████

In [ ]:
import pandas as pd
import json
from tqdm.notebook import tqdm 
import requests
from os import path, makedirs, rename, remove, listdir, walk
import time
import random

In [ ]:
from PyPaperBot.NetInfo import NetInfo
from PyPaperBot.HTMLparsers import getSchiHubPDF, SciHubUrls

In [ ]:
with open('drive/MyDrive/materialsperovskites.json') as fi:
  data = json.load(fi)

In [ ]:
tmp = pd.DataFrame(data)

In [ ]:
tmp = tmp.loc[tmp['type']=='journal-article'].sort_values(by='is-referenced-by-count',ascending=False)

In [ ]:
def URLjoin(*args):
    return "/".join(map(lambda x: str(x).rstrip('/'), args))

def getSaveDir(folder, fname):
    fname = fname.replace('/','.')
    fname = fname+".pdf"
    dir_ = path.join(folder, fname)

    return dir_


def getSaveDir2(folder, fname):
    #fname = fname.replace('/','.')
    fname = fname+".pdf"
    dir_ = path.join(folder, fname)

    return dir_


def setSciHubUrl():
    r = requests.get(NetInfo.SciHub_URLs_repo, headers=NetInfo.HEADERS)
    links = SciHubUrls(r.text)
    found = False

    for l in links:
        try:
            r = requests.get(l, headers=NetInfo.HEADERS)
            if r.status_code == 200:
                found = True
                NetInfo.SciHub_URL = l
                break
        except:
            pass
    if found:
        print("\nUsing {} as Sci-Hub instance".format(NetInfo.SciHub_URL))
    else:
        print("\nNo working Sci-Hub instance found!\nIf in your country Sci-Hub is not available consider using a VPN or a proxy")
        NetInfo.SciHub_URL = "https://sci-hub.st"

def saveFile(file_name,content):
    makedirs(path.dirname(file_name),exist_ok=True)
    f = open(file_name, 'wb')
    f.write(content)
    f.close()



In [ ]:
def download_paper(doi, files, dir_path = 'drive/MyDrive/papers/peroviskites/'):
  save_path = getSaveDir2(dir_path,doi)
  if not save_path in files:
    try:
      url = URLjoin(NetInfo.SciHub_URL, doi)
      r = requests.get(url, headers=NetInfo.HEADERS)
      content_type = r.headers.get('content-type')
      if 'application/pdf' not in content_type:
        #time.sleep(random.randint(1,5))
        pdf_link = getSchiHubPDF(r.text)
        if(pdf_link != None):
          r = requests.get(pdf_link, headers=NetInfo.HEADERS)
          content_type = r.headers.get('content-type')

      if 'application/pdf' in content_type:
        saveFile(save_path,r.content)
      else:
        return(doi)

    except:
      return(doi)
  else:
    pass
    #print("alread exists "+doi)

In [ ]:
PATH = 'drive/MyDrive/papers/peroviskites/'
files = [path.join(dp, f) for dp, dn, filenames in walk(PATH) for f in filenames if path.splitext(f)[1] == '.pdf']

In [ ]:
len(files)

41680

In [ ]:
if NetInfo.SciHub_URL==None:
  setSciHubUrl()

fails = []

for doi in tqdm(tmp.DOI):
  r = download_paper(doi, files)
  if (r):
    fails.append(r)


Using https://sci-hub.ee as Sci-Hub instance


  0%|          | 0/53452 [00:00<?, ?it/s]

In [ ]:
len(fails)

11811